In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

/Users/paek-injin/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# 하이퍼 파라미터 : 
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# 최적화 단계
## 하나의 에폭은 학습과 검증 단계 두 부분으로 구성됨
## 학습 단계 : 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴함
## 검증 단계 : 모델 성능이 개선되고 있는지 확인하기 위해 테스트 데이터셋을 반복함

In [5]:
# 손실함수
## MSELoss : 회귀문제에서
## NLLLoss : 분류문제에서
## CrossEntropyLoss : LogSoftmax + NLLLoss 

loss_fn = nn.CrossEntropyLoss() 
# 모델의 출력 로짓을 nn.CrossEntropyLoss에 전달하여 로짓을 정규화하고 예측 오류를 계산함

In [6]:
# 옵티마이저
## 최적화 : 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
## 최적화 알고리즘 : 이 과정이 수행되는 방식을 정의함
## 모든 최적화 절차는 optimizer 객체에 캡슐화됨.
## SGD, Adam, RMSProp 등의 다양한 옵티마이저 존재

### 옵티마이저 공부 필요

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

## 학습 단계에서 최적화는 3단계로 이뤄짐
# 1) optimizer.zero_grad() : 모델 매개변수의 변화도를 재설정함, 
    ## 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
# 2) loss.backwards() : 예측 손실을 역전파 -> pytorch는 각 매개변수에 대한 손실의 변화도를 저장함
# 3) 변화도 계산 뒤 optimizer.step() : 역전파 단계에서 수집된 변화도로 매개변수를 조정함

In [9]:
# 전체 구현
## 최적화 코드를 반복하여 수행하는 train_loop와
## 테스트 데이터로 모델의 성능을 측정하는 test_loop 정의

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측과 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        # 역전파 ## 최적화 3단계 세트로 발생!!
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch & 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss : {loss:>7f}    [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():    # 변화도 추적 멈추기!
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \nAccuracy : {(100*correct):0.1f}%, Avg loss : {test_loss:>8f}\n")

In [10]:
# 실행

loss_fn = nn.CrossEntropyLoss() # 손실함수 정의
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) # 옵티마이저 정의
epochs = 5 # 에폭 수 정의

for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    # 한 에폭에 train 1, test 1
print('Done!')

Epoch 1
--------------------------
loss : 2.174798    [    0/60000]
loss : 2.159283    [   64/60000]
loss : 2.173306    [  128/60000]
loss : 2.176456    [  192/60000]
loss : 2.161710    [  512/60000]
loss : 2.152644    [  576/60000]
loss : 2.156478    [  640/60000]
loss : 2.154219    [  704/60000]
loss : 2.158785    [ 1024/60000]
loss : 2.156526    [ 1088/60000]
loss : 2.159743    [ 1152/60000]
loss : 2.170592    [ 1216/60000]
loss : 2.151095    [ 1536/60000]
loss : 2.136799    [ 1600/60000]
loss : 2.131881    [ 1664/60000]
loss : 2.168006    [ 1728/60000]
loss : 2.160715    [ 8192/60000]
loss : 2.100732    [ 8256/60000]
loss : 2.116729    [ 8320/60000]
loss : 2.137813    [ 8384/60000]
loss : 2.123846    [ 8704/60000]
loss : 2.136239    [ 8768/60000]
loss : 2.166329    [ 8832/60000]
loss : 2.134322    [ 8896/60000]
loss : 2.116143    [ 9216/60000]
loss : 2.137441    [ 9280/60000]
loss : 2.111784    [ 9344/60000]
loss : 2.152424    [ 9408/60000]
loss : 2.134560    [ 9728/60000]
loss : 2